<a href="https://colab.research.google.com/github/yorkjong/stock-reports/blob/main/notebooks/ibd_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install and Setup (this section will be executed automatically)

#### Install Required Packages

In [4]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/ranking_utils"
%pip install requests-cache

!wget -O NotoSansTC-Regular.ttf https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?sign=bATsZP5QZdI_2EM15sAbcAE48Cacle91CpwUNOCMuM8=:0

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/ranking_utils) to /tmp/pip-req-build-qc9xvz97
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-qc9xvz97
  Running command git checkout -b feature/ranking_utils --track origin/feature/ranking_utils
  Switched to a new branch 'feature/ranking_utils'
  Branch 'feature/ranking_utils' set up to track remote branch 'feature/ranking_utils' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit 218c412e3209db0d43b106152a548273ed42543f
  Preparing metadata (setup.py) ... done
--2024-10-07 14:17:53--  https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?sign=bATsZP5QZdI_2EM15sAbcAE48Cacle91CpwUNOCMuM8=:0
Resolving share.cole.tw (share.cole.tw)... 180.218.0.193
Connecting to share.cole.tw (share.cole.tw)|180.218.0.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7110560 (6.8M) [

#### Setup and Configuration

In [5]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [6]:
# @title Set Chinese Font for matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fm.fontManager.addfont('NotoSansTC-Regular.ttf')
font_name = 'Noto Sans TC'
if font_name not in plt.rcParams['font.sans-serif']:
    plt.rcParams['font.sans-serif'].insert(0, font_name)

In [7]:
# @title ReadOnlyGitHub
import requests
import pandas as pd
from io import StringIO

class ReadOnlyGitHub:
    def __init__(self, repo_owner, repo_name, dir='', branch='main'):
        dir = dir.strip('/')
        base = 'https://raw.githubusercontent.com'
        if dir:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}/{dir}'
        else:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}'

        base = 'https://api.github.com/repos'
        if dir:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents'

    def file_exists(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        response = requests.head(url)
        return response.status_code == 200

    def list_filenames(self, dir_path=''):
        url = f'{self.api_url}/{dir_path}'
        response = requests.get(url)
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        elif response.status_code == 404:
            print(f"Directory not found: {url}")
            return []
        else:
            print(f"Request failed: {response.status_code} - {response.text}")
            return []

    def download_csv(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        if self.file_exists(file_path):
            return pd.read_csv(url)
        else:
            return pd.DataFrame()

# Create a GitHub instance
github = ReadOnlyGitHub(
    repo_owner='yorkjong',
    repo_name='stock-reports',
    dir='data/ibd'
)

In [16]:
# @title Taiwan Stock Name Lookup

class StockNameLookup:
    _df = None  # Class-level variable to hold the DataFrame

    @classmethod
    def _load_data(cls):
        if cls._df is None:  # Check if the DataFrame is already loaded
            gh = ReadOnlyGitHub(
                repo_owner='yorkjong',
                repo_name='stock-reports',
                dir='data/stock_list',
            )
            cls._df = gh.download_csv('taiwan_stock_OpenAPI.csv')

    @classmethod
    def tw_stock_name(cls, ticker):
        cls._load_data()  # Ensure data is loaded before accessing

        code = ticker.split('.')[0]  # Extract the code part

        # Filter the DataFrame to find the stock name for the given code
        stock_name = cls._df.loc[cls._df['Code'] == code, 'Name']

        # Check if the stock_name is empty and return an appropriate message
        if not stock_name.empty:
            return stock_name.values[0]  # Return the first matched stock name
        else:
            return None  # Return None if ticker not found


def tw_stock_name(ticker):
    return StockNameLookup.tw_stock_name(ticker)

In [8]:
# @title DataFrame Utilities

def print_column(df, column):
    if column in df.columns:
        print(', '.join(df[column]))

In [28]:
# @title Source of Tickers

def tickers_from_df(df):
    if 'Name' in df.columns:
        return [name.strip() for names in df['Name']
                for name in names.split(',')]
    elif 'Ticker' in df.columns:
        return [ticker.strip() for tickers in df['Ticker']
                for ticker in tickers.split(',')]
    return []

def major_indices():
    return ['^DJI', '^IXIC', '^NDX', '^RUT', '^SOX',
            '^TWII', '^N225', '^HSI',
            '^STOXX50E', '^FTSE', '^GDAXI', '^FCHI', '^GSPTSE']

def sector_indices():
    return ['SOXX', 'DVY',
            'IWB','IWM', 'IWV',  'IJR',
            'ITB', 'IHI', 'IYC', 'ITA', 'IAK',
            'IYZ', 'IYT', 'IYR', 'IYF', 'IYJ',
            'IYG', 'IYH', 'IYK', 'IDU', 'IYE', 'IHE',
            'IAT', 'IAI', 'IEO', 'IYM', 'IHF']

In [10]:
# @title Checkboxes

import ipywidgets as widgets

def cbs_create(symbols, n_pre_checked=10):
    '''Create a list of checkboxes'''
    return [
        widgets.Checkbox(
            value=(i < n_pre_checked),  # Set first n items as checked
            description=symbol,
            layout=widgets.Layout(width='auto'),
            style={'description_width': 'auto'}
        )
        for i, symbol in enumerate(symbols)
    ]

def cbs_with_grid(checkboxes, n_cols=5):
    '''Create a grid layouting the given checkboxes'''
    return widgets.GridBox(checkboxes, layout=widgets.Layout(
        width='auto',
        grid_template_columns=f'repeat({n_cols}, 1fr)',
        grid_gap='10px'  # Add some space between the checkboxes
    ))

def cbs_get_selected(checkboxes):
    '''Get the selected symbols from the given checkboxes'''
    return [checkbox.description for checkbox in checkboxes if checkbox.value]

def cbs_unselect_all(checkboxes):
    '''Unselect all checkboxes in the given list'''
    for checkbox in checkboxes:
        checkbox.value = False

def cbs_select_top(checkboxes, n=10):
    '''Select the top n checkboxes in the given list'''
    for i, checkbox in enumerate(checkboxes):
        checkbox.value = (i < n)

In [11]:
# @title Dropdown Menus

def create_period_dropdown(value='2y'):
    return widgets.Dropdown(
        options=['1y', '2y', '5y'],
        value=value,
        description='Period:',
    )

def create_interval_dropdown(value='1wk'):
    return widgets.Dropdown(
        options=['1d', '1wk'],
        value=value,
        description='Interval:',
    )

def create_rs_window_dropdown(value='3mo'):
    return widgets.Dropdown(
        options=['3mo', '12mo'],
        value=value,
        description='rs_window:',
    )

def create_style_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['default', 'classic', 'yahoo', 'charles', 'tradingview', 'binance', 'binancedark', 'mike', 'nightclouds', 'checkers', 'ibd', 'sas', 'starsandstripes', 'kenan', 'blueskies', 'brasil'],
        value='yahoo' if value is None else value,
        description='Style:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

def create_template_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['plotly', 'plotly_white', 'plotly_dark', 'ggplot2', 'seaborn', 'simple_white', 'presentation', 'xgridoff', 'ygridoff'],
        value='plotly_dark' if value is None else value,
        description='Template:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

In [12]:
# @title Multiple Searchable Dropdown Menus
import ipywidgets as widgets

def create_search_box():
    '''Create a Text widget for search input'''
    return widgets.Text(
        description='Search:',
        placeholder='Type to search',
        layout=widgets.Layout(width='auto')
    )

def create_dropdown(options, description='Stock:'):
    '''Create a Dropdown widget for displaying filtered options'''
    return widgets.Dropdown(
        description=description,
        options=[None] + options,  # None as the default option
        layout=widgets.Layout(width='auto'),
        value=None  # Set default value to None
    )

def update_dropdown(change, dropdown, options):
    '''Update the options in the dropdown based on search input'''
    search_text = change['new'].lower()
    filtered = [option for option in options if search_text in option.lower()]
    if filtered:
        dropdown.options = [None] + filtered
        dropdown.value = filtered[0]  # Auto-select the first matching option
    else:
        dropdown.options = [None]  # Retain only the None option if no match

def remove_duplicates_preserve_order(lst):
    '''Remove duplicates from a list while preserving order'''
    seen = set()
    result = []
    for item in lst:
        if item and item not in seen:
            seen.add(item)
            result.append(item)
    return result

def get_dropdowns_selected_options(dropdowns):
    '''Get selected options from the dropdowns'''
    selected = [dropdown.value for dropdown in dropdowns if dropdown.value]
    return remove_duplicates_preserve_order(selected)

def create_search_dropdowns(options, max_selections):
    '''Create a layout with search boxes and dropdowns'''
    # Create UI components
    search_boxes = [create_search_box() for _ in range(max_selections)]
    dropdowns = [create_dropdown(options) for _ in range(max_selections)]

    # Bind search box and dropdown menu events
    for search_box, dropdown in zip(search_boxes, dropdowns):
        search_box.observe(lambda change, dropdown=dropdown:
                            update_dropdown(change, dropdown, options),
                            names='value')
    # Create the layout
    controls = [widgets.HBox([search_box, dropdown])
                for search_box, dropdown  in zip(search_boxes, dropdowns)]
    layout = widgets.VBox(controls)
    return dropdowns, layout

In [13]:
# @title Outputs
outputs = widgets.VBox()

In [14]:
# @title enable_plotly_in_cell
# ref. https://stackoverflow.com/questions/76593068/plotly-figure-not-rendering-in-ipywidgets-interact-function-google-colab
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

### Glossary of Terms

#### Source (The source of stocks to analyze)
- This could include stocks traded on exchanges or components of a specific index.
- Common abbreviation(s) for the exchange or market sector:
  - For Taiwan Markets, possible values include:
    - **`TWSE`**: Taiwan Stock Exchange (台灣上市股票交易所)
    - **`TPEX`**: Taipei Exchange (上櫃交易所)
    - **`ESB`**: Emerging Stock Board (興櫃交易所)
  - Can also be combined with `+` (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`).
  - For America Markets, possible values include:
    - **`SPX`**: S&P 500 (標普五百指數)
    - **`DJIA`**: Dow Jones Industrial Average (道瓊指數)
    - **`NDX`**: NASDAQ-100 (納斯達克一百指數)
    - **`SOX`**: PHLX Semiconductor Index (費半指數)
  - Multiple indices can be combined using `+` (e.g., `SPX+DJIA+NDX+SOX`).

#### Period (Historical Data Time Range)
- The time range for which to fetch historical data.
  - **`2y`**: 2 years
  - **`6mo`**: 6 months

#### RS Window (Window for RS Calculation)
- The time window ('3mo' or '12mo') for Relative Strength:
  - **`3mo`**: 3 months
  - **`12mo`**: 12 months

#### RS (Relative Strength)
- Relative Strength (RS) is a metric used to evaluate the performance of a stock relative to a benchmark index.
  - A higher RS rating indicates that the stock has outperformed the index, while a lower RS rating suggests underperformance.
  - A value of 100 represents the performance of the benchmark index or market.
- The IBD RS calculates the performance over the last year, with the most recent quarter weighted double.
- The IBD 3-month RS calculates the performance over the last quarter.

#### RS Rating
- RS Rating, ranging from 1 (worst) to 99 (best), evaluates the price performance of a stock relative to a benchmark index.

### Execute Actions Step by Step


In [15]:
# @title Step 1. Pick a File

import ipywidgets as widgets

with requests_cache.disabled():
    filenames = github.list_filenames()

# Extract and sort all unique values in reverse order
all_dates = sorted(set(fn.split('_')[3].replace('.csv', '') for fn in filenames), reverse=True)
all_sources = sorted(set(fn.split('_')[0] for fn in filenames))
all_types = sorted(set(fn.split('_')[1] for fn in filenames), reverse=True)

# Calculate the maximum length of filenames to set dropdown width
if filenames:
    max_filename_length = max(len(fn) for fn in filenames)
else:
    max_filename_length = 0
dropdown_width = f'{max_filename_length * 10}px'  # 10px width per character

# Create dropdowns with dynamic width
date_dropdown = widgets.Dropdown(
    options=all_dates,
    description='Date:',
    layout=widgets.Layout(width=dropdown_width)
)

source_dropdown = widgets.Dropdown(
    options=all_sources,
    description='Source:',
    layout=widgets.Layout(width=dropdown_width)
)

type_dropdown = widgets.Dropdown(
    options=all_types,
    description='Type:',
    layout=widgets.Layout(width=dropdown_width)
)

# Create file selection dropdown
file_dropdown = widgets.Dropdown(
    options=[],
    description='File:',
    layout=widgets.Layout(width=dropdown_width)
)

def update_dropdowns(*args):
    # Filter files based on selected date
    date_filtered_files = [fn for fn in filenames if date_dropdown.value in fn]

    # Update Source dropdown
    available_sources = sorted(set(fn.split('_')[0] for fn in date_filtered_files))
    source_dropdown.options = available_sources
    if source_dropdown.value not in available_sources:
        source_dropdown.value = available_sources[0] if available_sources else None

    # Update Type dropdown
    available_types = sorted(set(fn.split('_')[1] for fn in date_filtered_files), reverse=True)
    type_dropdown.options = available_types
    if type_dropdown.value not in available_types:
        type_dropdown.value = available_types[0] if available_types else None

    # Update file options
    update_file_options()

def update_file_options(*args):
    filtered_files = [
        fn for fn in filenames
        if (date_dropdown.value in fn and
            source_dropdown.value == fn.split('_')[0] and
            type_dropdown.value == fn.split('_')[1])
    ]
    file_dropdown.options = filtered_files
    if filtered_files:
        file_dropdown.value = filtered_files[0]  # Set initial value to the first match
    else:
        file_dropdown.value = None

# Bind event handlers
date_dropdown.observe(update_dropdowns, 'value')
source_dropdown.observe(update_file_options, 'value')
type_dropdown.observe(update_file_options, 'value')

# Display all dropdowns
display(date_dropdown, source_dropdown, type_dropdown, file_dropdown)

# Initialize dropdowns
update_dropdowns()

Dropdown(description='Date:', layout=Layout(width='400px'), options=('ibd3mo',), value='ibd3mo')

Dropdown(description='Source:', layout=Layout(width='400px'), options=('TWSE+TPEX', 'U.S.Listed'), value='TWSE…

Dropdown(description='Type:', layout=Layout(width='400px'), options=('stocks',), value='stocks')

Dropdown(description='File:', layout=Layout(width='400px'), options=(), value=None)

### Step 2. Filter Stocks

In [ ]:
# @title Filter 1. Sorting {"run":"auto"}
cond0 = "(df['RS'] > 0)" # @param ["(df['RS'] > 0)"]
cond1 = "& (df['Rating (RS)'] > 95)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond2 = "& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))" # @param ["& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))", ""]
cond3 = "& (df['EPS RS (%)'] > 0)" # @param ["& (df['EPS RS (%)'] > 0)", "& (df['EPS RS (%)'] > 50)", "& (df['EPS RS (%)'] > 100)", "& (df['EPS RS (%)'] > 200)", "& (df['EPS RS (%)'] > 500)", ""]
cond4 = "& (df['Rev RS (%)'] > 0)" # @param ["& (df['Rev RS (%)'] > 0)", "& (df['Rev RS (%)'] > 5)", "& (df['Rev RS (%)'] > 10)", "& (df['Rev RS (%)'] > 20)", "& (df['Rev RS (%)'] > 50)", "& (df['Rev RS (%)'] > 100)", ""]
cond5 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond6 = "& (df['Price'] < 100)" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
sorted_column = "RS" # @param ["RS","1 Week Ago","1 Month Ago","3 Months Ago","6 Months Ago","Price","MA10","MA30","Volume / VMA10","EPS RS (%)", "Rev RS (%)"]
ascending = False # @param {"type":"boolean"}
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 7
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

selected_file = file_dropdown.value
source, kind, period, rs_win, date = selected_file.split('_')
rs_win = rs_win.replace('ibd', '')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nRS window {rs_win}\nDate: {date}")

df = github.download_csv(selected_file)
df = df[eval(cond)]
df = df.sort_values(by=sorted_column, ascending=ascending)
df_top_f1 = df.head(num_items)
display(df_top_f1)

print_column(df_top_f1, 'Name')
print_column(df_top_f1, 'Ticker')

In [ ]:
# @title Filter 2. Increasing RS > 0 {"run":"auto"}
cond0 = "(df['RS'] > 0)" # @param ["(df['RS'] > 0)"]
cond1 = "& (df['RS'] > df['1 Week Ago'])" # @param ["& (df['RS'] > df['1 Week Ago'])", ""]
cond2 = "& (df['1 Week Ago'] > df['1 Month Ago'])" # @param ["& (df['1 Week Ago'] > df['1 Month Ago'])", "& (df['RS'] > df['1 Month Ago'])", ""]
cond3 = "& (df['1 Month Ago'] > df['3 Months Ago'])" # @param ["& (df['1 Month Ago'] > df['3 Months Ago'])", ""]
cond4 = "& (df['3 Months Ago'] > df['6 Months Ago'])" # @param ["& (df['3 Months Ago'] > df['6 Months Ago'])", ""]
cond5 = "& (df['6 Months Ago'] > df['9 Months Ago'])" # @param ["& (df['6 Months Ago'] > df['9 Months Ago'])", ""]
cond6 = "& (df['Rating (RS)'] > 80)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond7 = "& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))" # @param ["& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))", ""]
cond8 = "& (df['EPS RS (%)'] > 0)" # @param ["& (df['EPS RS (%)'] > 0)", "& (df['EPS RS (%)'] > 50)", "& (df['EPS RS (%)'] > 100)", "& (df['EPS RS (%)'] > 200)", "& (df['EPS RS (%)'] > 500)", ""]
cond9 = "& (df['Rev RS (%)'] > 0)" # @param ["& (df['Rev RS (%)'] > 0)", "& (df['Rev RS (%)'] > 5)", "& (df['Rev RS (%)'] > 10)", "& (df['Rev RS (%)'] > 20)", "& (df['Rev RS (%)'] > 50)", "& (df['Rev RS (%)'] > 100)", ""]
cond10 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond11 = "& (df['Price'] < 100)" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 12
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

selected_file = file_dropdown.value
source, kind, period, rs_win, date = selected_file.split('_')
rs_win = rs_win.replace('ibd', '')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nRS window {rs_win}\nDate: {date}")

df = github.download_csv(selected_file)
df = df[eval(cond)]
df_top_f2 = df.head(num_items)
display(df_top_f2)

print_column(df_top_f2, 'Name')
print_column(df_top_f2, 'Ticker')

In [ ]:
# @title Filter 3. RS Breakout {"run":"auto"}
base = 0 # @param [0, 30, 60, 100] {"type":"raw"}
cond0 = "(df['RS'] > base)" # @param ["(df['RS'] > base)"]
cond1 = "" # @param ["& (df['1 Week Ago'] < base)",""]
cond2 = "& (df['1 Month Ago'] < base)" # @param ["& (df['1 Month Ago'] < base)",""]
cond3 = "& (df['3 Months Ago'] < base)" # @param ["& (df['3 Months Ago'] < base)",""]
cond4 = "& (df['6 Months Ago'] < base)" # @param ["& (df['6 Months Ago'] < base)",""]
cond5 = "& (df['9 Months Ago'] < base)" # @param ["& (df['9 Months Ago'] < base)",""]
cond6 = "& (df['Rating (RS)'] > 80)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond7 = "& (df['Volume / VMA10'] > 2)" # @param ["& (df['Volume / VMA10'] > 1.5)", "& (df['Volume / VMA10'] > 2)", "& (df['Volume / VMA10'] > 3)", "& (df['Volume / VMA10'] > 4)", ""]
cond8 = "& (df['Price'] > df['MA30'])" # @param ["& (df['Price'] > df['MA10'])", "& (df['Price'] > df['MA30'])", ""]
cond9 = "" # @param ["& (df['MA10'] > df['MA30'])",""]
cond10 = "& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))" # @param ["& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))", ""]
cond11 = "" # @param ["& (df['EPS RS (%)'] > 0)", "& (df['EPS RS (%)'] > 50)", "& (df['EPS RS (%)'] > 100)", "& (df['EPS RS (%)'] > 200)", "& (df['EPS RS (%)'] > 500)", ""]
cond12 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond13 = "" # @param ["& (df['Rev RS (%)'] > 0)", "& (df['Rev RS (%)'] > 5)", "& (df['Rev RS (%)'] > 10)", "& (df['Rev RS (%)'] > 20)", "& (df['Rev RS (%)'] > 50)", "& (df['Rev RS (%)'] > 100)", ""]
cond14 = "" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", "& (df['Price'] < 500)", ""]
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 15
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

selected_file = file_dropdown.value
source, kind, period, rs_win, date = selected_file.split('_')
rs_win = rs_win.replace('ibd', '')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nRS window {rs_win}\nDate: {date}")

df = github.download_csv(selected_file)
df = df[eval(cond)]
df_top_f3 = df.head(num_items)
display(df_top_f3)

print_column(df_top_f3, 'Name')
print_column(df_top_f3, 'Ticker')

In [32]:
# @title Filter 4. Groupby "Industry" {"run":"auto"}
cond0 = "(df['RS'] > 0)" # @param ["(df['RS'] > 0)"]
cond1 = "& (df['Rating (RS)'] > 95)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
num_items = 1 # @param [1, 2, 3, 4, 5] {"type":"raw"}

NUM_CONDS = 2

from vistock.ranking_utils import append_ratings, groupby_industry

selected_file = file_dropdown.value
source, kind, period, rs_win, date = selected_file.split('_')
rs_win = rs_win.replace('ibd', '')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nRS window {rs_win}\nDate: {date}")

stock_df = github.download_csv(selected_file)

# Filter out rows with NaN in the 'Ticker' column
stock_df = stock_df[stock_df['Ticker'].notna()]

rs_columns = ['RS',
              '1 Month Ago', '3 Months Ago', '6 Months Ago']
if 'Name' in stock_df.columns:
    columns = ['Sector', 'Ticker', 'Name'] + rs_columns
else:
    columns = ['Sector', 'Ticker'] + rs_columns
df = groupby_industry(stock_df, columns, key='RS')

df = df.sort_values(by='RS', ascending=False)
df = append_ratings(df, rs_columns)

cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))
df = df[eval(cond)]
df_top_f4 = df.head(num_items)
display(df_top_f4)

print_column(df_top_f4, 'Name')
print_column(df_top_f4, 'Ticker')

Source: TWSE+TPEX
Type: stocks
Period: 1y
RS window 3mo
Date: 20241007


IndexError: index 0 is out of bounds for axis 0 with size 0

### Step 3. Visualize Filtered Stocks

In [34]:
# @title Plot 1. IBD Relative Strength Comparison {"run":"auto"}
source = "Filter 4. Groupby Industry" # @param ["Filter 1. Sorting", "Filter 2. Increasing RS","Filter 3. RS Breakout", "Filter 4. Groupby Industry", "Major Global Stock Indices", "Sector Indices"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import matplotlib.pyplot as plt
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

MAX_STOCK_SELECTION = 10
N_COLS = 8  # the number of columns of the grid layout for checkboxes

symbols = {
    'Filter 1. Sorting': lambda x=df_top_f1: tickers_from_df(x),
    'Filter 2. Increasing RS': lambda x=df_top_f2: tickers_from_df(x),
    'Filter 3. RS Breakout': lambda x=df_top_f3: tickers_from_df(x),
    'Filter 4. Groupby Industry': lambda x=df_top_f4: tickers_from_df(x),
    'Major Global Stock Indices': major_indices,
    'Sector Indices': sector_indices,
}[source]()

checkboxes = cbs_create(symbols, MAX_STOCK_SELECTION)
checkbox_grid = cbs_with_grid(checkboxes, N_COLS)
btn_unselect_all = widgets.Button(description="Unselect All")
btn_unselect_all.on_click(lambda b: cbs_unselect_all(checkboxes))
btn_select_top = widgets.Button(description="Select Top 10")
btn_select_top.on_click(lambda b: cbs_select_top(checkboxes, MAX_STOCK_SELECTION))

period_dropdown = create_period_dropdown()
interval_dropown = create_interval_dropdown()
rs_window_dropdown = create_rs_window_dropdown('12mo')

cmp_theme_dropdown = {
    'mplfinance': create_style_dropdown('Comparison Theme:', 'charles'),
    'Plotly': create_template_dropdown('Comparison Theme:', 'plotly_dark'),
}[backend]
btn_plot = widgets.Button(description="Generate Plot")

out_msg, out_fig = widgets.Output(), widgets.Output()

ui = widgets.VBox([
    checkbox_grid,
    widgets.HBox([btn_unselect_all, btn_select_top]),
    widgets.VBox([period_dropdown, interval_dropown, rs_window_dropdown]),
    widgets.VBox([cmp_theme_dropdown, btn_plot]),
    out_msg, out_fig
])
display(ui)

import vistock.mpl as mpl
import vistock.plotly as ply

rs_cmp = {
    'mplfinance': mpl.ibd_rs_cmp,
    'Plotly': ply.ibd_rs_cmp,
}[backend]

def on_checkbox_change(change):
    selected_count = sum([cb.value for cb in checkboxes])
    if selected_count > MAX_STOCK_SELECTION:
        # Uncheck the last checked box if selection exceeds limit
        changed_checkbox = change['owner']
        changed_checkbox.value = False
        with out_msg:
            out_fig.clear_output()
            print(f"Only {MAX_STOCK_SELECTION} stocks can be selected at most.")

# Bind the checkbox change event to the function
for checkbox in checkboxes:
    checkbox.observe(on_checkbox_change, names='value')

def on_plot_click(b):
    symbols = cbs_get_selected(checkboxes)
    if not symbols:
        with out_msg:
            out_fig.clear_output()
            print("No stocks selected. Please select at least one stock.")
        return
    with out_fig:
        out_msg.clear_output()
        clear_output()
        interval = interval_dropown.value
        period = period_dropdown.value
        rs_window = rs_window_dropdown.value
        if rs_cmp is mpl.ibd_rs_cmp:
            rs_cmp.plot(symbols, interval=interval, period=period,
                        rs_window=rs_window,
                        style=cmp_theme_dropdown.value,
                        color_cycle=plt.cm.Paired.colors)
        else: # Plotly
            rs_cmp.plot(symbols, interval=interval, period=period,
                        rs_window=rs_window,
                        template=cmp_theme_dropdown.value,
                        colorway=px.colors.qualitative.Set3)

btn_plot.on_click(on_plot_click)

if backend == 'Plotly':
    enable_plotly_in_cell()


In [35]:
# @title Plot 2. IBD Stock Chart {"run":"auto"}
source = "Filter 4. Groupby Industry" # @param ["Filter 1. Sorting", "Filter 2. Increasing RS","Filter 3. RS Breakout","Filter 4. Groupby Industry","Major Global Stock Indices", "U.S. Listed Stocks", "Taiwan Stocks"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import functools as ft
import ipywidgets as widgets
from IPython.display import display, clear_output
import yfinance as yf

import vistock.stock_indices as si
import vistock.tw as tw

symbols = {
    'Filter 1. Sorting': lambda x=df_top_f1: tickers_from_df(x),
    'Filter 2. Increasing RS': lambda x=df_top_f2: tickers_from_df(x),
    'Filter 3. RS Breakout': lambda x=df_top_f3: tickers_from_df(x),
    'Filter 4. Groupby Industry': lambda x=df_top_f4: tickers_from_df(x),
    'Major Global Stock Indices': major_indices,
    'U.S. Listed Stocks': ft.partial(si.get_tickers, 'USLS'),
    'Taiwan Stocks': ft.partial(si.get_tickers, 'TWSE+TPEX+ESB'),
}[source]()
if source == 'Taiwan Stocks':
    symbols = [f"{tw_stock_name(s)} {s}" for s in symbols]

dropdowns, layout = create_search_dropdowns(symbols, 1)

period_dropdown2 = create_period_dropdown('2y')
interval_dropown2 = create_interval_dropdown('1wk')
rs_window_dropdown2 = create_rs_window_dropdown('12mo')

stock_theme_dropdown = {
    'mplfinance': create_style_dropdown('Stock Theme:', 'yahoo'),
    'Plotly': create_template_dropdown('Stock Theme:', 'plotly'),
}[backend]

btn_plot_prc = widgets.Button(description="Price/RS/Volume Chart",
                              layout=widgets.Layout(width='168px'))
btn_plot_fin = widgets.Button(description="Financial Chart",
                              layout=widgets.Layout(width='168px'))
btn_report_q = widgets.Button(description="Quarterly Report",
                              layout=widgets.Layout(width='168px'))
btn_report_a = widgets.Button(description="Annual Report",
                              layout=widgets.Layout(width='168px'))
btn_clear_last = widgets.Button(description="Clear Last")
btn_clear_all = widgets.Button(description="Clear All")
#outputs = widgets.VBox()

ui = widgets.VBox([
    layout,
    period_dropdown2, interval_dropown2, rs_window_dropdown2, stock_theme_dropdown,
    widgets.HBox([btn_plot_prc, btn_plot_fin]),
    widgets.HBox([btn_report_q, btn_report_a]),
    widgets.HBox([btn_clear_last, btn_clear_all]),
    outputs
])
display(ui)

import vistock.mpl as mpl
import vistock.plotly as ply

stock_chart = {
    'mplfinance': mpl.ibd_rs,
    'Plotly': ply.ibd_rs,
}[backend]

def get_symbols():
    symbols = get_dropdowns_selected_options(dropdowns)
    if source == 'Taiwan Stocks':
        symbols = [s.split()[0] for s in symbols]
    return symbols

def on_plot_prc_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        interval = interval_dropown2.value
        period = period_dropdown2.value
        rs_window = rs_window_dropdown2.value
        if not selected:
            print("No Stock Selected!")
        elif stock_chart is mpl.ibd_rs:
            stock_chart.plot(selected[0], interval=interval, period=period,
                             rs_window=rs_window,
                             style=stock_theme_dropdown.value,
                             legend_loc='upper left')
        else: # Plotly
            stock_chart.plot(selected[0], interval=interval, period=period,
                             rs_window=rs_window,
                             template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

import vistock.mpl.financials as mpl_fin
import vistock.plotly.financials as ply_fin

fin_chart = {
    'mplfinance': mpl_fin,
    'Plotly': ply_fin,
}[backend]

def on_plot_fin_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        elif fin_chart is mpl_fin:
            fin_chart.plot(selected[0], style=stock_theme_dropdown.value)
        else: # Plotly
            fin_chart.plot(selected[0], template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

def on_report_q_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        else:
            symbol = tw.as_yfinance(selected[0])
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Quarterly Financials:")
            display(yf.Ticker(ticker).quarterly_financials)
    outputs.children = (new_output,) + outputs.children

def on_report_a_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        else:
            symbol = tw.as_yfinance(selected[0])
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Annual Financials:")
            display(yf.Ticker(ticker).financials)
    outputs.children = (new_output,) + outputs.children

def on_clear_last_click(b):
    if outputs.children:
        children = list(outputs.children)
        children.pop(0)
        outputs.children = tuple(children)

def on_clear_all_click(b):
    outputs.children = ()

btn_plot_prc.on_click(on_plot_prc_click)
btn_plot_fin.on_click(on_plot_fin_click)
btn_report_q.on_click(on_report_q_click)
btn_report_a.on_click(on_report_a_click)
btn_clear_last.on_click(on_clear_last_click)
btn_clear_all.on_click(on_clear_all_click)

if backend == 'Plotly':
    enable_plotly_in_cell()


NameError: name 'df_top_f1' is not defined